### This scraper functions as follows:
1) With setup_driver, it launches the driver and sets it up
<span style="color:red">Would be interesting to already have the links ready for scraping
considering you just have to add sth like """?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng"""
at the end.
</span>
2) With setup_url_site, it uses some filters for language and to have chronological order
3) With single_page scraper, it scrapes all the required info on a single page
4) With scrape_all_pages, it takes the link of the first page after filters (they affect the link)
<span style="color:red"> The link can be simply altered to go to the next page</span>



### Question
- QUANTITY of pages (already 2900pages for pwc) ==> 1/5
- Should I use link alteration or click? Behaviour human?
- What is an acceptable rate of scraping (request/sec) so that I don't get banned?
- Should I do most hiring HEC companies?
- For how many years should I scrape?
- Should I only scrape belgian salaries then? Bc I cannot do that for reviews. For different jobs maybe do dimensionality reduction. OR check latent semantic analysis.
- I took only english is that okay?
- Should I pull one df per company? And retrain each time? 
- Sentiment of pros and cons? what can I get out of that?

### IDEA: 
- Try scraping salaries and see outcome for most hiring companies of HEC
- What about trying to cluster people by ratings etc, maybe I could reject the eternally unhappy (rageux) and the eternally happy (n'ont jamais vu autre chose) to just keep the relevant ones
- Check relationship between sentence length or vocabulary diversity and score?
- Check words that correlate the most with rating and do word clouds for every year.

In [2]:
# Imports
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, ElementNotInteractableException, WebDriverException
from getting_link import getting_company_reviews_urls_bs

import random
import time
import pandas as pd
import numpy as np

### Getting the companies links

In [4]:
companies = ['pwc', 'belfius', 'ey', 'bnp paribas', 'john cockerill', 
             'uliege','odoo', 'alter domus', 'nrb', 'cbc', 'kpmg',
             'total energies', 'kbc', 'bdo', 'deloitte', 'ethias']

In [44]:
# Etendre au lux et faire peut-être plus d'entreprises....

In [12]:
#companies_links = getting_company_reviews_urls_bs(companies)

In [16]:
df_links = pd.read_csv('data/company_links.csv')
df_links.head()

,Unnamed: 0,company,link
0,0,pwc,https://fr.glassdoor.be/Avis/PwC-Avis-E8450.htm
1,1,belfius,https://fr.glassdoor.be/Avis/Belfius-Bank-Avis...
2,2,ey,https://fr.glassdoor.be/Avis/EY-Avis-E2784.htm
3,3,bnp paribas,https://fr.glassdoor.be/Avis/BNP-Paribas-Avis-...
4,4,uliege,https://fr.glassdoor.be/Avis/Universidade-de-L...


In [43]:
df_links.tail()

,Unnamed: 0,company,link
6,6,kpmg,https://fr.glassdoor.be/Avis/KPMG-Avis-E2867.htm
7,7,total energies,https://fr.glassdoor.be/Avis/TotalEnergies-Avi...
8,8,kbc,https://fr.glassdoor.be/Avis/KBC-Avis-E10308.htm
9,9,bdo,https://fr.glassdoor.be/Avis/BDO-Avis-E7508.htm
10,10,deloitte,https://fr.glassdoor.be/Avis/Deloitte-Avis-E27...


In [ ]:
'PwC-Belgium-Reviews-EI_IE8450.0,3_IL.4,11_IN25'

In [ ]:
'Deloitte-Belgium-Reviews-EI_IE2763.0,8_IL.9,16_IN25'

In [37]:
# Set up
def setup_driver():
    """
    Launches driver, maximizes window size
    """
    driver = webdriver.Chrome('/Users/corentin/Documents/chromedriver')
    time.sleep(random.uniform(2, 4))
    # Setting window size to make sure the scrolls get to where they are supposed to
    driver.set_window_size(1920, 1080)
    driver.implicitly_wait(20)
    return driver

In [38]:
filter_url = '.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=PART_TIME&filter.employmentStatus=REGULAR&filter.employmentStatus=INTERN'
def get_company_nofilter(url):
    htm_index = url.index('.htm')
    url_nofilter = url[:htm_index]
    return url_nofilter

In [39]:
def get_rid_overlay(driver):
    """
    Sometimes the site shows an overlay that cannot be removed by clicking, which blocks interaction with the site,
    this is some code to remove it.
    """
    driver.execute_script("""
        javascript:(function(){
          document.getElementsByClassName('hardsellOverlay')[0].remove();
          document.getElementsByTagName("body")[0].style.overflow = "scroll";
          let style = document.createElement('style');
          style.innerHTML = `
            #LoginModal {
              display: none!important;
            }
          `;
          document.head.appendChild(style);
          window.addEventListener("scroll", function (event) {
            event.stopPropagation();
          }, true);
        })();
        """)

In [40]:
def get_link_page(url, page):
    url_nofilter = get_company_nofilter(url)
    return url_nofilter + '_P' + str(page) + filter_url

In [ ]:
def get_link(url):
    driver = setup_driver()
    driver.get(url)
    time.sleep(random.uniform(3, 5))
    set_site_language()
    time.sleep(random.uniform(3, 5))

    
    get_belgian_reviews()
    


### Initializing variables and scraping functions

In [119]:
# Initializating variables
subrating_classes = {'css-xd4dom': '*',
               'css-18v8tui': '**',
               'css-vl2edp': '***',
               'css-1nuumx7': '****',
               'css-s88v13': '*****'}
shape_styles = {'css-10xv9lv-svg': 'O',
         'css-1kiw93k-svg': 'X',
         'css-1h93d4v-svg': '-',
         'css-hcqxoa-svg': 'V'}

In [86]:
def get_rid_overlay(driver):
    """
    Sometimes the site shows an overlay that cannot be removed by clicking, which blocks interaction with the site,
    this is some code to remove it.
    """
    driver.execute_script("""
        javascript:(function(){
          document.getElementsByClassName('hardsellOverlay')[0].remove();
          document.getElementsByTagName("body")[0].style.overflow = "scroll";
          let style = document.createElement('style');
          style.innerHTML = `
            #LoginModal {
              display: none!important;
            }
          `;
          document.head.appendChild(style);
          window.addEventListener("scroll", function (event) {
            event.stopPropagation();
          }, true);
        })();
        """)

In [129]:
# Set up
def setup_driver():
    """
    Launches driver, maximizes window size
    """
    driver = webdriver.Chrome('/Users/corentin/Documents/chromedriver')
    time.sleep(random.uniform(2, 4))
    # Setting window size to make sure the scrolls get to where they are supposed to
    driver.set_window_size(1920, 1080)
    driver.implicitly_wait(20)
    return driver

In [88]:
# Set site language to us
def set_site_language():
    time.sleep(random.uniform(2, 4))
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(random.uniform(2, 4))
    language_choice = driver.find_element_by_xpath('//*[@id="Footer"]/nav/ul[2]/li[3]/div/div/div[1]')
    language_choice.click()
    soup = BeautifulSoup(driver.page_source, 'lxml')
    language_click_box = soup.find('div', class_='dropdownOptions dropdownExpanded animated above')
    for li in language_click_box.find_all('li'):
        country = li.find_all('span')[1].string
        if country == 'United States':
            us_id = li.get('id')
            break
    time.sleep(random.uniform(2, 4))
    us = driver.find_element_by_id(us_id)
    us.click()

In [89]:
# Get only english reviews
def get_english_reviews():
    time.sleep(random.uniform(2, 4))
    filter_button = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[2]/div[2]/button/span')
    filter_button.click()
    time.sleep(random.uniform(2, 4))
    language_button = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[2]/div/div[1]')
    language_button.click()
    time.sleep(random.uniform(2, 4))
    english = driver.find_element_by_xpath('//*[@id="option_eng"]')
    english.click()

In [90]:
# Start with most recent reviews
def orderby_recent():
    time.sleep(random.uniform(2, 4))
    driver.execute_script('window.scrollTo(0, 450)')
    time.sleep(random.uniform(2, 4))
    freshness_button = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[2]/div/span/div/div/div[1]')
    freshness_button.click()
    time.sleep(random.uniform(2, 4))
    most_recent = driver.find_element_by_xpath('//*[@id="option_DATE"]')
    most_recent.click()
    time.sleep(random.uniform(2, 4))

In [91]:
# Only belgian reviews
def get_belgian_reviews():
    time.sleep(random.uniform(2, 4))
    filter_button = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[2]/div[2]/button/span')
    filter_button.click()
    time.sleep(random.uniform(2, 4))
    location_box = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[1]/div/div[1]')
    location_box.click()
    time.sleep(random.uniform(2, 4))
    location_input = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[1]/div/div[1]/div/div/div/input')
    location_input.send_keys('Belgium')
    belgium = driver.find_element_by_xpath('//*[@id="option_N,25"]/span')
    belgium.click()

In [92]:
# For Interns and full time and part time
def interns_and_fulltime():
    time.sleep(random.uniform(2, 4))
    filter_button = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[2]/div[2]/button/span')
    filter_button.click()
    time.sleep(random.uniform(2, 4))
    jobs_category = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[3]/div[1]/div[2]/div/div[1]')
    jobs_category.click()
    time.sleep(random.uniform(2, 4))
    jobs_categories = driver.find_element_by_xpath('//*[@id="option_INTERN"]/div')
    jobs_categories.click()

In [109]:
def setup_url_site(driver, url):
    """
    Gets the url, sets the site in english, set the reviews in english, sets the 
    reviews to chronological order, gets only belgian reviews for intern, full time and part time jobs.
    """
    driver.get(url)
    functions = [set_site_language, get_english_reviews, orderby_recent, get_belgian_reviews, interns_and_fulltime]
    
    for func in functions:
        max_tries = 5
        tries = 0
        if tries <= max_tries:
            try:
                func()
            except (ElementClickInterceptedException, ElementNotInteractableException, WebDriverException):
                get_rid_overlay(driver)
                tries +=1
                func()
        else:
            print('Max tries for setup attained')
    
    # Returns modified url with all filters
    url = driver.current_url
    return driver, url

In [94]:
def scrape_ratings(post):
    """
    Returns a string with the main rating and a list with subrating
    """
    # Main rating
    main_rating = post.find('span', class_ = 'ratingNumber mr-xsm').string
    
    # Subratings
    try:
        rating_categories = post.find('div', class_ = 'content')
        ratings = rating_categories.find_all('li') # Each rating has a different class, it's the only way to differentiate

        subratings = {}
        for subrating in ratings:
            #Get categories, it's not always 6 of them
            subrating_category = subrating.find_all('div')[0].string
            subrating_class = subrating.find_all('div')[1]['class'][0]
            subrating = subrating_classes[subrating_class]
            subratings[subrating_category] = subrating
    except:
        subratings = {'Work/Life Balance': np.nan,
                      'Culture & Values': np.nan, 
                      'Diversity & Inclusion': np.nan, 
                      'Career Opportunities': np.nan,
                      'Compensation and Benefits': np.nan,
                      'Senior Management': np.nan}
    
    return main_rating, subratings

In [95]:
def single_page_scraper(html): #Needs the driver.page_source
    """
    Scrapes rating, subrating, post_title, date_and_job, recommend, CEO approval,
    business outlook, pros and cons of all postings on a single page and
    returns a dataframe
    """
    #Initializing df
    df = pd.DataFrame({'main_rating': [],
                       'Work/Life Balance': [], 
                       'Culture & Values': [], 
                       'Diversity & Inclusion': [], 
                       'Career Opportunities': [],
                       'Compensation and Benefits': [],
                       'Senior Management': []
                       })
    soup = BeautifulSoup(html, 'lxml')
    
    # Get postings on a single page
    # First and other nine reviews have different classes
    first_posting = soup.find_all('li', class_ = 'empReview cf pb-0 mb-0')
    next_postings = soup.find_all('li', class_ = 'noBorder empReview cf pb-0 mb-0')
    # Extending first_posting with the nine other and renaming the variable adequatly
    first_posting.extend(next_postings)
    postings = first_posting
    
    
    # Looping on postings
    for post in postings:
        # Get status
        status = post.find('span', class_ = 'pt-xsm pt-md-0 css-1qxtz39 eg4psks0').string
        # Get general opinion 
        post_title = post.find('h2', class_ = 'mb-xxsm mt-0 css-93svrw el6ke055').string
        # Get date and job title
        date_and_job = post.find('span', class_ = 'authorInfo').text        
        # Get pros and cons
        pros = post.find('span', {'data-test': 'pros'}).string
        cons = post.find('span', {'data-test': 'cons'}).string
        # Get shapes for Recommend, CEO approval, Business outlook for a single post
        other_scores = {'Recommend': '',
                       'CEO Approval': '',
                       'Business Outlook': ''}
        bar_shapes = post.find('div', class_ = 'd-flex my-std reviewBodyCell recommends css-1y3jl3a e1868oi10')
        for i, element in  enumerate(bar_shapes.find_all('svg')):
            other_scores[list(other_scores)[i]] = shape_styles[element['class'][1]]
        #Initating row for dataframe 
        post_data = {'date_and_job': date_and_job,
                     'post_title': post_title,
                     'status': status,
                     'pros': pros,
                     'cons': cons,
                     'Recommend': other_scores['Recommend'],
                     'CEO Approval': other_scores['CEO Approval'],
                     'Business Outlook': other_scores['Business Outlook']}
        # Getting ratings
        main_rating, subratings = scrape_ratings(post)
        post_data['main_rating'] = main_rating
        for key, value in subratings.items():
            post_data[key] = value
             
        #Populating the dataframe
        df = df.append((post_data),
                        ignore_index=True)
    return df

In [114]:
def url_decomposer(url):
    """
    Takes any url, and returns the domain, the filter part of the url and the company code part of the url, 
    with these you can then recompose the link of any page of that company's reviews as such:
    domain_page_url + company_code + _IPx + filter_url with x the page in question
    """
    domain_page_url = 'https://www.glassdoor.com/Reviews/'
    
    filter_index = url.index('.htm')
    filter_url = url[filter_index:]
    
    try:
        page_index = url.index('_IP')
        page_url = url[page_index:]
    except ValueError:
        page_url = ''
    
    company_code = url.replace(page_url, '').replace(domain_page_url, '').replace(filter_url, '')
    return domain_page_url,  filter_url, company_code

In [130]:
def scrape_all_pages(driver, url): # Would like to be able to scrape 5 years back, you can try to use date in date_and_job column
    """
    Takes link of first setted (english, recent) reviews, scrapes it and 
    goes automatically to next pages scrapes them etc
    """
    
    df = pd.DataFrame()
    domain_page_url,  filter_url, company_code = url_decomposer(url)
    for p in range(3):
        try:
            driver.get(url)
            time.sleep(random.uniform(2, 4))
            df_p = single_page_scraper(driver.page_source)
            df = df.append(df_p, ignore_index=True)
            time.sleep(random.uniform(7, 13))
            url = domain_page_url + company_code + '_IP' + str(p) + filter_url
        except WebDriverException:
            get_rid_overlay(driver)
            time.sleep(random.uniform(2, 4))
            df_p = single_page_scraper(driver.page_source)
            df = df.append(df_p, ignore_index=True)
            time.sleep(random.uniform(7, 13))
            url = domain_page_url + company_code + '_IP' + str(p) + filter_url
        
    return df

In [140]:
driver = setup_driver()

In [141]:
driver, url = setup_url_site(driver, url)

JavascriptException: Message: javascript error: Cannot read properties of undefined (reading 'remove')
  (Session info: chrome=100.0.4896.127)


In [128]:
df = scrape_all_pages(driver, driver.current_url)

### Scraping the reviews for all companies

In [ ]:
driver = setup_driver()
for row in companies_links:
    company, link = row['company'], row['link']
    

In [148]:
domain_page_url,  filter_url, company_code = url_decomposer(deloitte)

In [150]:
domain_page_url2,  filter_url2, company_code2 = url_decomposer(pwc)

In [151]:
domain_page_url + company_code2 + '_IP2' + filter_url

'https://www.glassdoor.com/Reviews/PwC-Reviews-E8450_IP2.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=PART_TIME&filter.employmentStatus=REGULAR&filter.employmentStatus=INTERN'

In [152]:
get_rid_overlay(driver)

In [149]:
pwc = 'https://www.glassdoor.com/Reviews/PwC-Reviews-E8450.htm'

In [153]:
pwc_belgium = 'https://www.glassdoor.com/Reviews/PwC-Belgium-Reviews-EI_IE8450.0,3_IL.4,11_IN25.htm?filter.iso3Language=eng&filter.employmentStatus=REGULAR&filter.employmentStatus=PART_TIME'

In [147]:
deloitte = 'https://www.glassdoor.com/Reviews/Deloitte-Belgium-Reviews-EI_IE2763.0,8_IL.9,16_IN25.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=PART_TIME&filter.employmentStatus=REGULAR&filter.employmentStatus=INTERN'

In [154]:
deloitte_belgium_only = 'https://www.glassdoor.com/Reviews/Deloitte-Belgium-Reviews-EI_IE2763.0,8_IL.9,16_IN25.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng'

In [155]:
linkaas = domain_page_url + 'Deloitte-Belgium-Reviews-EI_IE2763.0,8_IL.9,16_IN25' + '_IP2' + filter_url

In [156]:
linkaas

'https://www.glassdoor.com/Reviews/Deloitte-Belgium-Reviews-EI_IE2763.0,8_IL.9,16_IN25_IP2.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=PART_TIME&filter.employmentStatus=REGULAR&filter.employmentStatus=INTERN'

In [36]:
get_link_page(df_links.link.iloc[0], 3)

'https://fr.glassdoor.be/Avis/PwC-Avis-E8450_P3.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=PART_TIME&filter.employmentStatus=REGULAR&filter.employmentStatus=INTERN'

In [26]:
link_page(df_links.link.iloc[0], 3)

'https://www.glassdoor.com/Reviews/PwC-Avis-E8450_P3.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=PART_TIME&filter.employmentStatus=REGULAR&filter.employmentStatus=INTERN'

'https://fr.glassdoor.be/Avis/PwC-Avis-E8450.htm'

In [28]:
link_us_belg = 'https://www.glassdoor.com/Reviews/PwC-Belgium-Reviews-EI_IE8450.0,3_IL.4,11_IN25.htm?filter.iso3Language=eng&filter.employmentStatus=REGULAR&filter.employmentStatus=PART_TIME'

In [29]:
link_page(link_us_belg, 3)

'https://www.glassdoor.com/Reviews/https://www.glassdoor.com/Reviews/PwC-Belgium-Reviews-EI_IE8450.0,3_IL.4,11_IN25_P3.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=PART_TIME&filter.employmentStatus=REGULAR&filter.employmentStatus=INTERN'